In [1]:
import json
import jax
import jax.numpy as jnp
import surrojax_gp
import oed_toolbox

In [2]:
def load_beam_data(file_dir):
    with open(file_dir, 'r') as f:
        data = json.load(f)
    try:
        x = jnp.stack([jnp.array(data[key]) for key in ('E', 'y_rot')], axis=-1)
        y = jnp.array(jnp.array(data['end_disp'])).reshape(-1,1)
    except KeyError:
        x = jnp.stack([jnp.array(data[key]) for key in ('E', 'Beam Angle')], axis=-1)
        y = jnp.array(jnp.array(data['End Displacement'])).reshape(-1,1)
    return {'x': x, 'y': y}

def scale_data(data):
    for key, val in data.items():
        data[key] = (val- jnp.min(val ,axis=0))/(jnp.max(val ,axis=0) - jnp.min(val ,axis=0))
    return data

linear_data = scale_data(load_beam_data('linear_beam_train.json'))
nonlinear_data = scale_data(load_beam_data('nonlinear_beam_train.json'))

In [3]:
def kernel(x_1, x_2, params):
    lengths = jnp.array([params[f"length_{i}"] for i in range(2)])
    inv_lengths = jnp.diag(lengths**(-1))
    ln_k_d = -0.5*(x_1 - x_2).T @ inv_lengths @ (x_1 - x_2) 
    return params["const"]*jnp.exp(ln_k_d)
constraints = {"length_0": {">": 10**-1, "<": 10**1}, 
               "length_1": {">": 10**-1, "<": 10**1}, 
               "const": {">": 10**-1, "<": 10**2}}
linear_gp = surrojax_gp.create_gp(kernel, linear_data['x'], linear_data['y'], constraints)
nonlinear_gp = surrojax_gp.create_gp(kernel, nonlinear_data['x'], nonlinear_data['y'], constraints)

-93.08633
-133.60971
-95.6327
-156.34497
-240.05603
-195.32083
-245.65036
-172.73447
-256.8095
-261.1317
-263.31085
-264.59674
-265.4391
-263.36307
-266.41797
-266.0668
-267.96014
-267.65726
-264.6313
-264.3997
-267.65726
-267.65726
-268.13397
-268.13397
-268.13397
-265.8656
-268.13397
-268.13397
-267.3597
-268.13397
-268.13397
-265.55286
-268.13397
-268.13397
-266.26984
-268.13397
-267.30255
-267.0605
-266.83017
-268.13397
-268.13397
-268.13397
-268.13397
-268.13397
-268.13397
-268.13397
-268.13397
-268.13397
-268.13397
-268.13397
-268.13397
-268.13397
-268.13397
-268.13397
-268.13397
      fun: array(-268.13397217)
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 0.07310563,  5.18304443, -4.95946884])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 55
      nit: 11
     njev: 55
   status: 0
  success: True
        x: array([44.64443157,  0.63340229,  0.70003374])
-141.74521
-134.26138
897.66437
-141.81198
-141.58661
-141.81198
-141.8

In [4]:
linear_beam = oed_toolbox.models.Model.from_surrojax_gp(linear_gp)
nonlinear_beam = oed_toolbox.models.Model.from_surrojax_gp(nonlinear_gp)

In [5]:
noise_cov = 0.05*jnp.identity(1)
prior_mean = 5*jnp.ones((1,))
prior_cov = 1.0*jnp.identity(1)

In [6]:
likelihood = oed_toolbox.uncertainties.Likelihood.from_model_plus_constant_gaussian_noise(linear_beam, noise_cov)
fisher_information = oed_toolbox.uncertainties.FisherInformation(likelihood)

In [19]:
fisher_information(0.2,0.5,100)

{'cov': array([[1.6500549]], dtype=float32),
 'cov_dd': array([[[13.298292]]], dtype=float32)}

In [40]:
minimizer = oed_toolbox.optim.gradient_descent_for_map()
ape_linearisation = \
oed_toolbox.losses.APE.from_model_plus_constant_gaussian_noise(model, minimizer, prior_mean, prior_cov, noise_cov)

NameError: name 'model' is not defined

In [ ]:
prior = oed_toolbox.uncertainties.Prior.gaussian(prior_mean, prior_cov)
joint = oed_toolbox.uncertainties.Joint.from_prior_and_likelihood(prior, likelihood)
posterior = oed_toolbox.uncertainties.Posterior.from_posteriax()
ape_full = oed_toolbox.losses.APE(joint, likelihood, posterior)